In [11]:
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph import graph
from typing_extensions import TypedDict, List
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class MultiToolState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

internet_search_tool = TavilySearch()

def add(a:int, b:int) -> int:
    """Adds two numbers.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The sum of the two numbers.
    """
    return a + b    

def multiply(a:int, b:int) -> int:
    """Multiplies two numbers.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The product of the two numbers.
    """
    return a * b

tools = [internet_search_tool, add, multiply]

llm_with_tools = llm.bind_tools(tools)

def superbot(state: MultiToolState):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder = StateGraph(MultiToolState)
graph_builder.add_node("superbot", superbot)
graph_builder.add_edge(START, "superbot")
graph_builder.add_edge("superbot", END) 

graph = graph_builder.compile()

res = graph.invoke({"messages": ["can you add 2 and 30, please?"]})

print(res)

#output 
#{'messages': [HumanMessage(content='can you add 2 and 30, please?', additional_kwargs={}, response_metadata={}, id='a0ff2120-91e9-44ed-a17d-00073b033685'), AIMessage(content='2 + 30 = 32', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 46, 'total_tokens': 54, 'completion_time': 0.002070092, 'prompt_time': 0.011436672, 'queue_time': 0.001603759, 'total_time': 0.013506764}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2477e04561', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6a60b6cc-0ef9-4bba-b9f7-5bda29a79007-0', usage_metadata={'input_tokens': 46, 'output_tokens': 8, 'total_tokens': 54})]}

{'messages': [HumanMessage(content='can you add 2 and 30, please?', additional_kwargs={}, response_metadata={}, id='f6b2a10c-9aaa-460f-a7fa-3bb2adc378ce'), AIMessage(content='2 + 30 = 32', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 46, 'total_tokens': 54, 'completion_time': 0.000916371, 'prompt_time': 0.023197879, 'queue_time': 0.083873946, 'total_time': 0.02411425}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_adc8919c18', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--93285b04-1104-4e78-bcb0-613c2e139da3-0', usage_metadata={'input_tokens': 46, 'output_tokens': 8, 'total_tokens': 54})]}


In [ ]:
res = graph.invoke({"messages": ["what is the date of latest news you have about AI?"]})

print(res)

#output
#{'messages': [HumanMessage(content='what is the date of latest news you have about AI?', additional_kwargs={}, response_metadata={}, id='659cb887-98ff-4de7-b724-89dc44ec4cd3'), AIMessage(content="My knowledge cutoff is currently December 2023, but I don't have have access to real-time news or updates. However, I can provide information on the latest developments in AI up to my knowledge cutoff date.\n\nAs of December 2023, some of the recent advancements in AI include:\n\n1. **Chatbots and Conversational AI**: Improved natural language processing (NLP) capabilities, enabling more human-like conversations and interactions.\n2. **Generative AI**: Advances in generative models, such as Generative Adversarial Networks (GANs) and Variational Autoencoders (VAEs), allowing for more realistic image and video generation.\n3. **Explainable AI (XAI)**: Increased focus on developing techniques to explain and interpret AI decision-making processes, improving transparency and trust in AI systems.\n4. **Edge AI**: Growing interest in deploying AI models on edge devices, such as smartphones, smart home devices, and autonomous vehicles, to reduce latency and improve real-time processing.\n5. **Quantum AI**: Ongoing research on the intersection of quantum computing and AI, exploring the potential for quantum computing to accelerate AI computations and solve complex problems.\n\nPlease note that these developments are based on my knowledge cutoff date, and there may have been significant advancements in AI since then. For the latest news and updates, I recommend checking reputable sources, such as academic journals, tech news websites, or official AI research institutions.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 290, 'prompt_tokens': 47, 'total_tokens': 337, 'completion_time': 0.649490382, 'prompt_time': 0.025722708, 'queue_time': 0.084673358, 'total_time': 0.67521309}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_adc8919c18', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--191f7032-a44b-41eb-8cc5-e7ace3e9f679-0', usage_metadata={'input_tokens': 47, 'output_tokens': 290, 'total_tokens': 337})]}

{'messages': [HumanMessage(content='what is the date of latest news you have about AI?', additional_kwargs={}, response_metadata={}, id='659cb887-98ff-4de7-b724-89dc44ec4cd3'), AIMessage(content="My knowledge cutoff is currently December 2023, but I don't have have access to real-time news or updates. However, I can provide information on the latest developments in AI up to my knowledge cutoff date.\n\nAs of December 2023, some of the recent advancements in AI include:\n\n1. **Chatbots and Conversational AI**: Improved natural language processing (NLP) capabilities, enabling more human-like conversations and interactions.\n2. **Generative AI**: Advances in generative models, such as Generative Adversarial Networks (GANs) and Variational Autoencoders (VAEs), allowing for more realistic image and video generation.\n3. **Explainable AI (XAI)**: Increased focus on developing techniques to explain and interpret AI decision-making processes, improving transparency and trust in AI systems.\n4

In [14]:
def superbot(state: MultiToolState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder = StateGraph(MultiToolState)
graph_builder.add_node("superbot", superbot)
graph_builder.add_edge(START, "superbot")
graph_builder.add_edge("superbot", END) 

graph = graph_builder.compile()

res = graph.invoke({"messages": ["can you add 2 and 30, please?"]})

print(res)

#output
#{'messages': [HumanMessage(content='can you add 2 and 30, please?', additional_kwargs={}, response_metadata={}, id='82e6eecd-d7ac-46f1-bf2a-a51f35470efc'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'rmwn2k9n8', 'function': {'arguments': '{"a":2,"b":30}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 1925, 'total_tokens': 1943, 'completion_time': 0.050792134, 'prompt_time': 0.155359789, 'queue_time': 0.068414258, 'total_time': 0.206151923}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_a2c262bc3a', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--acf11271-23f8-4aa1-9248-593420e17c23-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 30}, 'id': 'rmwn2k9n8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1925, 'output_tokens': 18, 'total_tokens': 1943})]}

{'messages': [HumanMessage(content='can you add 2 and 30, please?', additional_kwargs={}, response_metadata={}, id='0cbe8a8d-4506-4c45-a167-ea3e02815885'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '23fedq0sd', 'function': {'arguments': '{"a":2,"b":30}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 1925, 'total_tokens': 1943, 'completion_time': 0.044072875, 'prompt_time': 0.151852196, 'queue_time': 0.002850214, 'total_time': 0.195925071}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2477e04561', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--71cf0926-0560-4edd-8d22-ffa48711330b-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 30}, 'id': '23fedq0sd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1925, 'output_tokens': 18, 'total_tokens': 1943})]}


In [ ]:
res = graph.invoke({"messages": ["what is the date of latest news you have about AI?"]})

print(res)

#output
#{'messages': [HumanMessage(content='what is the date of latest news you have about AI?', additional_kwargs={}, response_metadata={}, id='fc7d21e6-ec06-4234-bf59-c51a7073691e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'bf077kmdg', 'function': {'arguments': '{"query":"latest AI news","time_range":"day","topic":"news"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 1926, 'total_tokens': 1957, 'completion_time': 0.09198286, 'prompt_time': 0.150059319, 'queue_time': 0.007860204, 'total_time': 0.242042179}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2477e04561', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6660d3bc-ca20-4b91-a066-b0eff44866e9-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'latest AI news', 'time_range': 'day', 'topic': 'news'}, 'id': 'bf077kmdg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1926, 'output_tokens': 31, 'total_tokens': 1957})]}

{'messages': [HumanMessage(content='what is the date of latest news you have about AI?', additional_kwargs={}, response_metadata={}, id='fc7d21e6-ec06-4234-bf59-c51a7073691e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'bf077kmdg', 'function': {'arguments': '{"query":"latest AI news","time_range":"day","topic":"news"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 1926, 'total_tokens': 1957, 'completion_time': 0.09198286, 'prompt_time': 0.150059319, 'queue_time': 0.007860204, 'total_time': 0.242042179}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2477e04561', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6660d3bc-ca20-4b91-a066-b0eff44866e9-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'latest AI news', 'time_range': 'day', 'topic': 'news'}, 'id': 'bf077kmdg', 'type': 'tool_call'}], usage_metadata={'input_to